In [1]:
import pandas as pd
import utils
import calcDistributions
%matplotlib qt

# Load and preprocess MID

In [2]:
trips, persons = calcDistributions.loadData()

---

# Distance-Distributions

## Home-Work

Reasons for Lognorm: 
  - Very basic distribution
  - Arises through central limit theorem then positive independent vars are multiplied
  - 4x as fast sampling in java (commons.math)
  - Tests:
    - Wins most regions 4/7
    - Wins on whole dataset ? TODO Weibull wins
    - Wins then KS-Stat is weighted by sample number

Reasons Weibull: 
  - Also common
  - Wins on the Sum of KS-Stat

All in all both seem to fit well

In [3]:
home_work = utils.getDistanceDistributions(trips, "H", "W")

## Home-School

Lognorm wins all

In [4]:
home_school = utils.getDistanceDistributions(trips, "H", "S")

## Somwhere-Shopping

Lognorm wins all

In [5]:
any_shopping = utils.getDistanceDistributions(trips, None, "P")

## Somewhere-Other

Lognorm wins all

In [6]:
any_other = utils.getDistanceDistributions(trips, None, "O")

---

# Activity Chains

Features:
- Daytype (7 Weekdays + Holiday))
- Hom. Group (Working, non-Working, Pupil)
- Mob. Group (Car-person, multimodal Person with Car, Non car owner)
- Age

In [8]:
groups = calcDistributions.getActivityChains(trips, persons)

In [9]:
df = pd.DataFrame( calcDistributions.getActivityChains(trips, persons))

In [10]:
df

,weekday,Hom. Grp,Mob. Grp,Age,count,Group chains
0,1,working,car_user,0_40,1499,"{'HWH': 339, 'HWHOH': 82, 'H': 81, 'HWOH': 60,..."
1,1,working,car_user,40_60,4492,"{'HWH': 1019, 'H': 264, 'HWHOH': 235, 'HOH': 1..."
2,1,working,car_user,60_100,1014,"{'HWH': 192, 'H': 81, 'HWHOH': 48, 'HOH': 44, ..."
3,1,working,car_user,undefined,7011,"{'HWH': 1551, 'H': 427, 'HWHOH': 365, 'HOH': 2..."
4,1,working,car_other,0_40,1165,"{'HWH': 233, 'HWHOH': 85, 'H': 53, 'HWOH': 49,..."
...,...,...,...,...,...,...
571,undefined,undefined,other,undefined,38683,"{'H': 7739, 'HOH': 3536, 'HWH': 2620, 'HPH': 1..."
572,undefined,undefined,undefined,0_40,92054,"{'H': 12299, 'HOH': 8249, 'HSH': 8000, 'HWH': ..."
573,undefined,undefined,undefined,40_60,97200,"{'HWH': 12659, 'H': 10615, 'HOH': 6938, 'HWHOH..."
574,undefined,undefined,undefined,60_100,111077,"{'H': 21013, 'HOH': 13421, 'HPH': 6904, 'HO': ..."


---

# Dwell time

## Get time data

In [10]:
trips = utils.addTime(trips)
grp = trips.groupby('HP_ID_Lok')
activityChains = (grp['StartLoc'].sum() + grp['DestLoc'].last())
dwellTimes = trips.pivot(index='HP_ID_Lok', columns='W_ID', values='DwellTime')

In [11]:
model = utils.getGaussianMixture("HOPH", activityChains, dwellTimes, plot=True)